In [1]:
# ipynotebook reload magic
%load_ext autoreload
%autoreload 2

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [1]:
from prismatic.vla.datasets import RLDSDataset, RLDSBatchTransform
from prismatic.models.action_heads import L1RegressionActionHead
from prismatic.models.projectors import ProprioProjector, NoisyActionProjector
from prismatic.models.backbones.llm.prompting import PurePromptBuilder
from transformers import AutoProcessor
import torch
from prismatic.vla.action_tokenizer import ActionTokenizer

# Initialize required variables
cfg = {
    "data_root_dir": "/slurm_home/byungjun_alinlab/tensorflow_datasets/",
    "dataset_name": "bridge_carrot_plate_dataset",
    "shuffle_buffer_size": 1000,
    "image_aug": True,
    "use_proprio": True,
    "use_diffusion": True,
    "use_l1_regression": True,
    "batch_size": 32,
    "grad_accumulation_steps": 1,
    "learning_rate": 1e-4,
    "lora_rank": 32,
    "lora_dropout": 0.0,
    "use_lora": True,
    "merge_lora_during_training": True,
    "use_val_set": True,
    "val_freq": 10,
    "max_steps": 1000,
    "save_freq": 100,
    "val_time_limit": 3600,
    "wandb_log_freq": 10,
    "diffusion_sample_freq": 50,
    "num_images_in_input": 2,
    "use_film": True,
    "resume": False,
    "resume_step": None,
    "run_id_override": None,
    "run_id_note": None,
    "wandb_entity": "your-wandb-entity",
    "wandb_project": "your-wandb-project",
    "vla_path": "/path/to/vla",
    "shuffle_buffer_size": 1000,
    "image_aug": True,
    "num_patches": 16,
    "num_diffusion_steps": 1000,
}

class DummyVLA:
    class Module:
        config = type('config', (object,), {"image_sizes": [224, 224]})
    module = Module()

vla = DummyVLA()

processor = AutoProcessor.from_pretrained("openvla/openvla-7b", trust_remote_code=True)
action_tokenizer = ActionTokenizer(processor.tokenizer)
batch_transform = RLDSBatchTransform(
    action_tokenizer=action_tokenizer,
    base_tokenizer=processor.tokenizer,
    image_transform=processor.image_processor.apply_transform,
    prompt_builder_fn=PurePromptBuilder,
    use_wrist_image=cfg["num_images_in_input"] > 1,
    use_proprio=cfg["use_proprio"],
)



/slurm_home/byungjun_alinlab/openvla-oft/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using LIBERO constants:
  NUM_ACTIONS_CHUNK = 8
  ACTION_DIM = 7
  PROPRIO_DIM = 8
  ACTION_PROPRIO_NORMALIZATION_TYPE = bounds_q99
If needed, manually set the correct constants in `prismatic/vla/constants.py`!


2025-04-11 19:22:13.117839: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-11 19:22:13.117994: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-11 19:22:13.120263: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-11 19:22:13.132648: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-11 19:22:14.974602: W tensorflow/compiler/tf2

In [2]:
train_dataset = RLDSDataset(
    cfg["data_root_dir"],
    cfg["dataset_name"],
    batch_transform,
    resize_resolution=tuple(vla.module.config.image_sizes),
    shuffle_buffer_size=cfg["shuffle_buffer_size"],
    image_aug=cfg["image_aug"],
    # split="train[:1%]"
)

print("Data loading function executed successfully.")


KeyError: 'bridge_carrot_plate_dataset'

In [4]:
len(train_dataset)

2135463

In [6]:
next(iter(train_dataset))["language_instruction"]

for sample in train_dataset:
    if "eggplant" in str(sample["language_instruction"]):
        print(sample["language_instruction"])

W0000 00:00:1744339145.845573 1581052 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "CropAndResize" attr { key: "T" value { type: DT_FLOAT } } attr { key: "extrapolation_value" value { f: 0 } } attr { key: "method" value { s: "bilinear" } } inputs { dtype: DT_FLOAT shape { dim { size: 1 } dim { size: 224 } dim { size: 224 } dim { size: -10 } } } inputs { dtype: DT_FLOAT shape { dim { size: -2 } dim { size: 4 } } } inputs { dtype: DT_INT32 shape { dim { size: -2 } } } inputs { dtype: DT_INT32 shape { dim { size: 2 } } } device { type: "CPU" vendor: "AuthenticAMD" model: "241" frequency: 2295 num_cores: 32 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 524288 l3_cache_size: 268435456 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { dim { size: -2 } dim { size: -14 } dim { size: -15 } dim { size: -10 } } }
W0000 00:00:174433914

b'put eggplant in pot or pan'
b'put eggplant in pot or pan'
b'put eggplant in pot or pan'
b'put eggplant in pot or pan'
b'put eggplant in pot or pan'
b'put eggplant in pot or pan'
b'put eggplant in pot or pan'
b'put eggplant in pot or pan'
b'put eggplant in pot or pan'
b'put eggplant in pot or pan'
b'put eggplant in pot or pan'
b'put eggplant in pot or pan'
b'put eggplant in pot or pan'
b'take the eggplant and put it between the two right burners'
b'take the eggplant and put it between the two right burners'
b'take the eggplant and put it between the two right burners'
b'take the eggplant and put it between the two right burners'
b'take the eggplant and put it between the two right burners'
b'take the eggplant and put it between the two right burners'
b'take the eggplant and put it between the two right burners'
b'take the eggplant and put it between the two right burners'
b'take the eggplant and put it between the two right burners'
b'take the eggplant and put it between the two right

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_22_device_/job:localhost/replica:0/task:0/device:CPU:0}} Dimension -2 must be >= 0
	 [[{{node BroadcastTo_2}}]] [Op:IteratorGetNext] name: 

In [7]:
import tensorflow_datasets as tfds

tfds.builder("bridge_v2", data_dir="/shared/dataset")

04/11 [12:45:08] INFO     | >> Load dataset info from /shared/dataset/bridge_v2/1.0.0           ]8;id=987444;file:///slurm_home/byungjun_alinlab/openvla-oft/.venv/lib/python3.9/site-packages/tensorflow_datasets/core/dataset_info.py\dataset_info.py]8;;\:]8;id=799871;file:///slurm_home/byungjun_alinlab/openvla-oft/.venv/lib/python3.9/site-packages/tensorflow_datasets/core/dataset_info.py#599\599]8;;\